In [2]:
import numpy as np
import pandas as pd
import nltk
import string
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [3]:
games = pd.read_csv('steam.csv', nrows = 10000)
#games

In [4]:
posters_df = pd.read_csv('steam_media_data.csv' nrows =1000)
posters_df

,steam_appid,header_image,screenshots,background,movies
0,10,https://steamcdn-a.akamaihd.net/steam/apps/10/...,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...",https://steamcdn-a.akamaihd.net/steam/apps/10/...,NaN
1,20,https://steamcdn-a.akamaihd.net/steam/apps/20/...,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...",https://steamcdn-a.akamaihd.net/steam/apps/20/...,NaN
2,30,https://steamcdn-a.akamaihd.net/steam/apps/30/...,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...",https://steamcdn-a.akamaihd.net/steam/apps/30/...,NaN
3,40,https://steamcdn-a.akamaihd.net/steam/apps/40/...,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...",https://steamcdn-a.akamaihd.net/steam/apps/40/...,NaN
4,50,https://steamcdn-a.akamaihd.net/steam/apps/50/...,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...",https://steamcdn-a.akamaihd.net/steam/apps/50/...,NaN
...,...,...,...,...,...
27327,1065230,https://steamcdn-a.akamaihd.net/steam/apps/106...,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...",https://steamcdn-a.akamaihd.net/steam/apps/106...,"[{'id': 256747452, 'name': 'Video', 'thumbnail..."
27328,1065570,https://steamcdn-a.akamaihd.net/steam/apps/106...,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...",https://steamcdn-a.akamaihd.net/steam/apps/106...,"[{'id': 256747487, 'name': 'Game Trailer', 'th..."
27329,1065650,https://steamcdn-a.akamaihd.net/steam/apps/106...,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...",https://steamcdn-a.akamaihd.net/steam/apps/106...,"[{'id': 256747530, 'name': 'Gameplay Trailer',..."
27330,1066700,https://steamcdn-a.akamaihd.net/steam/apps/106...,"[{'id': 0, 'path_thumbnail': 'https://steamcdn...",https://steamcdn-a.akamaihd.net/steam/apps/106...,"[{'id': 256747638, 'name': 'Main Trailer', 'th..."


In [3]:
descriptions = pd.read_csv('steam_description_data.csv')
#description

## Data preparation

In [4]:
descriptions = descriptions.rename(columns={'steam_appid': 'appid'})
#descriptions

In [5]:
games = games.merge(descriptions, on = 'appid')
games.head()

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,...,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price,detailed_description,about_the_game,short_description
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,...,0,124534,3339,17612,317,10000000-20000000,7.19,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...,Play the world's number 1 online action game. ...
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,...,0,3318,633,277,62,5000000-10000000,3.99,One of the most popular online action games of...,One of the most popular online action games of...,One of the most popular online action games of...
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,...,0,3416,398,187,34,5000000-10000000,3.99,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...,Enlist in an intense brand of Axis vs. Allied ...
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,...,0,1273,267,258,184,5000000-10000000,3.99,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...,Enjoy fast-paced multiplayer gaming with Death...
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,...,0,5250,288,624,415,5000000-10000000,3.99,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...,Return to the Black Mesa Research Facility as ...


In [6]:
# Replace 'selected_column' with the actual list of column names you want to select
selected_columns = ['appid','name', 'developer', 'categories', 'genres', 'steamspy_tags', 'price', 'detailed_description']
games = games[selected_columns]
games.head(5)


,appid,name,developer,categories,genres,steamspy_tags,price,detailed_description
0,10,Counter-Strike,Valve,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,7.19,Play the world's number 1 online action game. ...
1,20,Team Fortress Classic,Valve,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,3.99,One of the most popular online action games of...
2,30,Day of Defeat,Valve,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,3.99,Enlist in an intense brand of Axis vs. Allied ...
3,40,Deathmatch Classic,Valve,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,3.99,Enjoy fast-paced multiplayer gaming with Death...
4,50,Half-Life: Opposing Force,Gearbox Software,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,3.99,Return to the Black Mesa Research Facility as ...


### Data processing

In [7]:
#checking missing data
games.isnull().sum()

appid                   0
name                    0
developer               0
categories              0
genres                  0
steamspy_tags           0
price                   0
detailed_description    0
dtype: int64

In [8]:
#checking duplicate data
games.duplicated().sum()

0

In [9]:
# Apply str.lower() to all columns with text data
games = games.apply(lambda x: x.str.lower() if x.dtype == 'object' else x)
games

,appid,name,developer,categories,genres,steamspy_tags,price,detailed_description
0,10,counter-strike,valve,multi-player;online multi-player;local multi-p...,action,action;fps;multiplayer,7.19,play the world's number 1 online action game. ...
1,20,team fortress classic,valve,multi-player;online multi-player;local multi-p...,action,action;fps;multiplayer,3.99,one of the most popular online action games of...
2,30,day of defeat,valve,multi-player;valve anti-cheat enabled,action,fps;world war ii;multiplayer,3.99,enlist in an intense brand of axis vs. allied ...
3,40,deathmatch classic,valve,multi-player;online multi-player;local multi-p...,action,action;fps;multiplayer,3.99,enjoy fast-paced multiplayer gaming with death...
4,50,half-life: opposing force,gearbox software,single-player;multi-player;valve anti-cheat en...,action,fps;action;sci-fi,3.99,return to the black mesa research facility as ...
...,...,...,...,...,...,...,...,...
9995,501830,the orb chambers ii,starsystemstudios™;jaryd brad spann,single-player;steam achievements;steam trading...,action;casual;indie;strategy,strategy;action;indie,0.79,<strong><u>warning: this game requires skill. ...
9996,501840,don't disturb,midnight party,single-player;steam trading cards,indie,indie;singleplayer;puzzle,2.69,"<img src=""https://steamcdn-a.akamaihd.net/stea..."
9997,501850,dcr: drive.crash.repeat,tarboosh games,single-player;steam achievements;steam trading...,adventure;casual;indie;racing,indie;casual;adventure,6.19,"<h2 class=""bb_tag"">what is the game?</h2><br><..."
9998,501860,slayer shock,minor key games,single-player;steam achievements;full controll...,action;indie;rpg;strategy,action;indie;rpg,14.99,"from david pittman (<i>eldritch</i>, <i>neon s..."


In [10]:
# Function to preprocess and tokenize text
def preprocess_text(text):
   # Remove punctuation
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)
    
    # Tokenize the text
    tokens = word_tokenize(text)
    
    return ' '.join(tokens)

# Apply preprocessing to the "detailed_description" column
games['detailed_description'] = games['detailed_description'].apply(preprocess_text)
games


,appid,name,developer,categories,genres,steamspy_tags,price,detailed_description
0,10,counter-strike,valve,multi-player;online multi-player;local multi-p...,action,action;fps;multiplayer,7.19,play the worlds number 1 online action game en...
1,20,team fortress classic,valve,multi-player;online multi-player;local multi-p...,action,action;fps;multiplayer,3.99,one of the most popular online action games of...
2,30,day of defeat,valve,multi-player;valve anti-cheat enabled,action,fps;world war ii;multiplayer,3.99,enlist in an intense brand of axis vs allied t...
3,40,deathmatch classic,valve,multi-player;online multi-player;local multi-p...,action,action;fps;multiplayer,3.99,enjoy fastpaced multiplayer gaming with deathm...
4,50,half-life: opposing force,gearbox software,single-player;multi-player;valve anti-cheat en...,action,fps;action;sci-fi,3.99,return to the black mesa research facility as ...
...,...,...,...,...,...,...,...,...
9995,501830,the orb chambers ii,starsystemstudios™;jaryd brad spann,single-player;steam achievements;steam trading...,action;casual;indie;strategy,strategy;action;indie,0.79,stronguwarning this game requires skill ustron...
9996,501840,don't disturb,midnight party,single-player;steam trading cards,indie,indie;singleplayer;puzzle,2.69,img srchttpssteamcdnaakamaihdnetsteamapps50184...
9997,501850,dcr: drive.crash.repeat,tarboosh games,single-player;steam achievements;steam trading...,adventure;casual;indie;racing,indie;casual;adventure,6.19,h2 classbbtagwhat is the gameh2brstrongdcr dri...
9998,501860,slayer shock,minor key games,single-player;steam achievements;full controll...,action;indie;rpg;strategy,action;indie;rpg,14.99,from david pittman ieldritchi ineon structi an...


In [11]:
# Combining relevant columns into a single column for TF-IDF vectorization
games['combined_features'] = games['categories'] + ' ' + games['genres'] + ' ' + games['developer']

In [12]:
# Creating a TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

# Fit and transform the TF-IDF vectorizer on the combined features
tfidf_matrix = tfidf_vectorizer.fit_transform(games['combined_features'])
tfidf_matrix

<10000x7417 sparse matrix of type '<class 'numpy.float64'>'
	with 122374 stored elements in Compressed Sparse Row format>

In [13]:
# Compute the cosine similarity between games based on TF-IDF vectors
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)


In [14]:
def get_game_recommendations(game_title, num_recommendations=10):
    # Get the index of the game in the dataset
    game_index = games.index[games['name'] == game_title].tolist()[0]

    # Get the cosine similarity scores for the game
    similar_scores = list(enumerate(cosine_sim[game_index]))

    # Sort the games by similarity score in descending order
    similar_games = sorted(similar_scores, key=lambda x: x[1], reverse=True)

    # Get the top N similar games
    top_similar_games = similar_games[1:num_recommendations + 1]

    # Extract the game titles
    recommended_games = [games.iloc[i[0]]['name'] for i in top_similar_games]

    return recommended_games

In [15]:
# Example usage:
user_searched_game = 'counter-strike'
recommendations = get_game_recommendations(user_searched_game)
if recommendations:
    print(f"Recommended games for '{user_searched_game}':")
    for game in recommendations:
        print(game)
else:
    print(f"Game '{user_searched_game}' not found in the dataset.")

Recommended games for 'counter-strike':
team fortress classic
deathmatch classic
ricochet
half-life
counter-strike: condition zero
day of defeat
half-life deathmatch: source
half-life 2: deathmatch
lost planet™: extreme condition
call of duty: black ops - mac edition


In [16]:
import pickle

In [17]:
pickle.dump(games.to_dict(),open('games_dict.pkl','wb'))

In [18]:
games['name'].values

array(['counter-strike', 'team fortress classic', 'day of defeat', ...,
       'dcr: drive.crash.repeat', 'slayer shock', 'star project'],
      dtype=object)

In [19]:
games.to_dict()

{'appid': {0: 10,
  1: 20,
  2: 30,
  3: 40,
  4: 50,
  5: 60,
  6: 70,
  7: 80,
  8: 130,
  9: 220,
  10: 240,
  11: 280,
  12: 300,
  13: 320,
  14: 340,
  15: 360,
  16: 380,
  17: 400,
  18: 420,
  19: 440,
  20: 500,
  21: 550,
  22: 570,
  23: 620,
  24: 630,
  25: 730,
  26: 1002,
  27: 1200,
  28: 1250,
  29: 1300,
  30: 1500,
  31: 1510,
  32: 1520,
  33: 1530,
  34: 1600,
  35: 1610,
  36: 1630,
  37: 1640,
  38: 1670,
  39: 1690,
  40: 1700,
  41: 1840,
  42: 1900,
  43: 1930,
  44: 2100,
  45: 2200,
  46: 2210,
  47: 2270,
  48: 2280,
  49: 2290,
  50: 2300,
  51: 2310,
  52: 2320,
  53: 2330,
  54: 2340,
  55: 2350,
  56: 2360,
  57: 2370,
  58: 2390,
  59: 2400,
  60: 2450,
  61: 2500,
  62: 2520,
  63: 2590,
  64: 2600,
  65: 2610,
  66: 2620,
  67: 2630,
  68: 2640,
  69: 2710,
  70: 2720,
  71: 2780,
  72: 2800,
  73: 2810,
  74: 2820,
  75: 2840,
  76: 2850,
  77: 2870,
  78: 2900,
  79: 2910,
  80: 2920,
  81: 2990,
  82: 3010,
  83: 3020,
  84: 3050,
  85: 3130,
  8

In [21]:
pickle.dump(cosine_sim,open('ifidf_matrix.pkl','wb'))